In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# EDA
1. combine daily datasets into a dictionary to concat together as a whole df to manage as a whole.
<br><br>
2. extract only the necessary columns to be used in prediction
      'Province_State', 
      'Last_Update', 
      'Confirmed', 
      'Deaths', 
      'Recovered', 
      'Active', 
      'Incident_Rate',
      'Mortality_Rate',
      'People_Tested',
      'Testing_Rate',
      'Hospitalization_Rate'
      
      To reduce space and time, I will use columns which I feel are MOST important: 
      'Province_State', 'Last_Update', 'Confirmed', 'Deaths', 'Recovered', 'Active',
      'People_Tested', 'Testing_Rate', 'Mortality_Rate', 'Incident_Rate'


In [2]:
import os

csv_list = []
# append csv file names into an empty list to be iterated through afterwards
for files in sorted(os.listdir("csse_covid19_daily_us/.")):
    if files.endswith(".csv"):
        csv_list.append(files)
        
# check if it appended correctly to csv_list
print(len(csv_list))
csv_list[:5]

100


['04-12-2020.csv',
 '04-13-2020.csv',
 '04-14-2020.csv',
 '04-15-2020.csv',
 '04-16-2020.csv']

In [3]:
# iterate through csv_list and read_csv > set dates as dict keys > daily data as vals
feature_list = ['Province_State', 'Last_Update', 'Confirmed', 'Deaths', 
                'Recovered', 'Active', 'People_Tested', 'Testing_Rate', 
                'Mortality_Rate', 'Incident_Rate']
 
# datetime_idx = pd.DatetimeIndex([filename[0:-4] for filename in csv_list])
daily_states_dict = {}

for i in range(len(csv_list)):
    csv_str = 'csse_covid19_daily_us/'+csv_list[i]
    today_df = pd.read_csv(csv_str, sep=',')
    filter_df = today_df[feature_list]
    daily_states_dict[i] = filter_df

if len(daily_states_dict) == 100:
    print("Appended all 100 days and corresponding dataframes.")
else:
    print("Loop didn't work correctly.")

Appended all 100 days and corresponding dataframes.


# Data pre-processing

In [4]:
# combine dictionary items into a whole df
df = pd.concat(daily_states_dict, axis=0)

# first, reset index with dates by re-formatting date column and groupby dates
df['date'] = pd.to_datetime(df['Last_Update']).dt.strftime('%Y-%m-%d')
df.drop('Last_Update', axis=1, inplace=True)
#df.set_index('date', inplace=True) doesn't groupby them, but sets date as index
df.head()


Province_State  Confirmed  Deaths  Recovered   Active  People_Tested  \
0  0         Alabama       3563      93        NaN   3470.0        21583.0   
   1          Alaska        272       8       66.0    264.0         8038.0   
   2         Arizona       3542     115        NaN   3427.0        42109.0   
   3        Arkansas       1280      27      367.0   1253.0        19722.0   
   4      California      22795     640        NaN  22155.0       190328.0   
...              ...        ...     ...        ...      ...            ...   
99 53       Virginia      78375    2031    10107.0  66237.0       920461.0   
   54     Washington      47743    1453        NaN  46290.0       809339.0   
   55  West Virginia       5084     100     3466.0   1518.0       234980.0   
   56      Wisconsin      43018     846    33130.0   9042.0       783866.0   
   57        Wyoming       2187      24     1652.0    511.0        44114.0   

       Testing_Rate  Mortality_Rate  Incident_Rate        date  
0  0     460.300152        2.610160      75.988020  2020-04-12  
   1    1344.711576        2.941176      45.504049  2020-04-12  
   2     578.522286        3.246753      48.662422  2020-04-12  
   3     761.753354        2.109375      49.439423  2020-04-12  
   4     485.423868        2.812020      58.137726  2020-04-12  
...             ...             ...            ...         ...  
99 53  10783.890236        2.591388     918.221845  2020-07-21  
   54  10628.369959        3.043378     626.968757  2020-07-21  
   55  13111.647649        1.966955     283.682086  2020-07-21  
   56  13462.857630        1.966619     738.831904  2020-07-21  
   57   7622.170886        1.097394     377.877493  2020-07-21  

[5818 rows x 10 columns]

In [5]:
# check len(Last_Update) are all equal
valct = df.groupby('Province_State')['date'].value_counts()

valct[valct > 1]

Province_State  date      
Florida         2020-04-14    2
Name: date, dtype: int64

    Florida has duplicated date on the 14th of April. This will affect our analysis

In [6]:
# go back to initial, pre-processed df
df[df.Province_State == 'Florida'].head()

,,Province_State,Confirmed,Deaths,Recovered,Active,People_Tested,Testing_Rate,Mortality_Rate,Incident_Rate,date
0,10,Florida,19895,461,NaN,19434.0,182753.0,860.718651,2.317165,93.700227,2020-04-12
1,11,Florida,21019,499,NaN,122520.0,196207.0,924.083459,0.405628,579.387193,2020-04-14
2,11,Florida,21628,571,NaN,21057.0,203180.0,956.924459,2.640096,101.862202,2020-04-14
3,11,Florida,22511,596,NaN,21915.0,213509.0,1005.571337,2.647595,106.020900,2020-04-15
4,11,Florida,23343,668,NaN,22675.0,219248.0,1032.600520,2.861672,109.939402,2020-04-16


* It can be assumed that the update for April 13th was posted slightly past midnight (based on the exact 'ns' format from original data), which led to being considered as next day. However, the column values for the 13th, except for 'confirmed' and one or two others, seems significantly different from its previous day and the next days. 
    
* I decided that I will move the midnight data to the 13th, and alter the values for sigficantly different values to Florida's avg for that corresponding week. 

In [7]:

# 1. locate index value
df_ = df.reset_index(drop=True)
df_[(df_.Province_State=='Florida') & (df_.date == '2020-04-14')]

# 2. change date value to 13th
df_.loc[df_.index==70, 'date'] = '2020-04-13'

# 3. get first week (12-19th, excl. 13th) avg for active, mortality and incident rates
wk1_fl = df_[(df_.Province_State == 'Florida') & (df_.index != 70)].reset_index(drop=True)
wk1_fl_sub = wk1_fl.loc[:6, ['Active','Mortality_Rate','Incident_Rate']].agg(np.mean)
wk1_fl_sub

# 4. fill in the incorrect column vals with corresponding series vals
df_.loc[df_.index==70, 'Active'] = wk1_fl_sub[0]
df_.loc[df_.index==70, 'Mortality_Rate'] = wk1_fl_sub[1]
df_.loc[df_.index==70, 'Incident_Rate'] = wk1_fl_sub[2]

# check result
df_.loc[63:68]
df_[df_.Province_State == 'Florida'].head()

,Province_State,Confirmed,Deaths,Recovered,Active,People_Tested,Testing_Rate,Mortality_Rate,Incident_Rate,date
10,Florida,19895,461,NaN,19434.000000,182753.0,860.718651,2.317165,93.700227,2020-04-12
70,Florida,21019,499,NaN,22771.285714,196207.0,924.083459,2.752916,110.303397,2020-04-13
129,Florida,21628,571,NaN,21057.000000,203180.0,956.924459,2.640096,101.862202,2020-04-14
188,Florida,22511,596,NaN,21915.000000,213509.0,1005.571337,2.647595,106.020900,2020-04-15
247,Florida,23343,668,NaN,22675.000000,219248.0,1032.600520,2.861672,109.939402,2020-04-16


##### Manage missing values
    - for dates, fillna(method='ffill') - depending on row placement of the state within the df

    - other feature values: confirmed, deaths, active fill with 0
    
    - if there are nulls in other rest feature vals, look over and fill with 0

In [8]:
# check count and dropna where most values are null
print(df_.info(), '\n')
print("number of NaNs: ", df_.isnull().sum().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5818 entries, 0 to 5817
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province_State  5818 non-null   object 
 1   Confirmed       5818 non-null   int64  
 2   Deaths          5818 non-null   int64  
 3   Recovered       4391 non-null   float64
 4   Active          5801 non-null   float64
 5   People_Tested   5600 non-null   float64
 6   Testing_Rate    5600 non-null   float64
 7   Mortality_Rate  5700 non-null   float64
 8   Incident_Rate   5600 non-null   float64
 9   date            5799 non-null   object 
dtypes: float64(6), int64(2), object(2)
memory usage: 454.7+ KB
None 

number of NaNs:  2235


In [9]:
# first, change to easy access names
df_.columns = df_.columns.str.lower()
df_.rename(columns={'province_state': 'state'}, inplace=True)


In [10]:
print(df_.state.unique())

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Diamond Princess' 'District of Columbia'
 'Florida' 'Georgia' 'Grand Princess' 'Guam' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Puerto Rico' 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee'
 'Texas' 'Utah' 'Vermont' 'Virginia' 'Washington' 'West Virginia'
 'Wisconsin' 'Wyoming' 'American Samoa' 'Northern Mariana Islands'
 'Recovered' 'Virgin Islands']


In [11]:
# assess value stat and compare to entire data
print("GRAND PRINCESS: \n\n{}".format(df_[df_['state']=='Grand Princess'].agg(np.mean)), '\n\n')

print("DIAMOND PRINCESS: \n\n{}".format(df_[df_['state']=='Diamond Princess'].agg(np.mean)), '\n')


GRAND PRINCESS: 

confirmed         103.000000
deaths              2.670000
recovered           0.000000
active            100.330000
people_tested            NaN
testing_rate             NaN
mortality_rate      2.592233
incident_rate            NaN
dtype: float64 


DIAMOND PRINCESS: 

confirmed         49.0
deaths             0.0
recovered          0.0
active            49.0
people_tested      NaN
testing_rate       NaN
mortality_rate     0.0
incident_rate      NaN
dtype: float64 



In [12]:
df_.describe()

,confirmed,deaths,recovered,active,people_tested,testing_rate,mortality_rate,incident_rate
count,5818.000000,5818.000000,4391.000000,5801.000000,5.600000e+03,5600.000000,5700.000000,5600.000000
mean,32692.493469,1668.792712,11057.379868,22955.531854,3.489471e+05,5864.633381,4.047353,481.614900
std,60430.019162,4109.291674,18960.665192,49303.290427,6.208745e+05,4634.567964,2.858028,460.975763
min,0.000000,0.000000,0.000000,-120720.000000,3.000000e+00,5.391708,0.000000,0.000000
25%,2311.500000,67.000000,724.500000,842.000000,4.889600e+04,2196.716493,2.390435,139.496918
50%,10975.000000,368.500000,3157.000000,6258.000000,1.433470e+05,4735.449294,3.780917,317.451194
75%,35629.500000,1422.750000,11837.000000,20340.000000,3.897520e+05,8440.463156,5.151574,673.463513
max,407326.000000,32506.000000,177871.000000,390100.000000,6.414321e+06,26549.442542,70.370370,2093.837730


    Comparing feature means to the entire dataset, Diamond & Grand cruise ship 
    circumstances may not be relevant to average numbers in US, so it may be 
    feasible to extract them, as well as 'Recovered', since that is not a state.

In [13]:
# drop cruise ships & other not relevant

filterdf = df_.set_index('state')
filterdf = filterdf.drop(['Grand Princess', 'Diamond Princess', 'Recovered'], axis=0)
filterdf.reset_index(inplace=True)
filterdf.head()

,state,confirmed,deaths,recovered,active,people_tested,testing_rate,mortality_rate,incident_rate,date
0,Alabama,3563,93,NaN,3470.0,21583.0,460.300152,2.610160,75.988020,2020-04-12
1,Alaska,272,8,66.0,264.0,8038.0,1344.711576,2.941176,45.504049,2020-04-12
2,Arizona,3542,115,NaN,3427.0,42109.0,578.522286,3.246753,48.662422,2020-04-12
3,Arkansas,1280,27,367.0,1253.0,19722.0,761.753354,2.109375,49.439423,2020-04-12
4,California,22795,640,NaN,22155.0,190328.0,485.423868,2.812020,58.137726,2020-04-12


    Glimpsing over the difference between given 'active' and 
    'confirmed'-('deaths'+'recovered') results('active_off'), there are some states 
    that doesn't match up, indicating that residual active numbers are neither
    reported in 'deaths' or 'recovered'. 
    
    Although that may be something to consider, it could also be a miniscule aspect in 
    predicting y.
    
    Therefore, 'recovered' nulls can be filled with 0, since many of them match up with 
    the confirmed and active differences.

In [14]:
clean_df = filterdf.copy()
clean_df['recovered'] = filterdf['recovered'].fillna(0)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           5600 non-null   object 
 1   confirmed       5600 non-null   int64  
 2   deaths          5600 non-null   int64  
 3   recovered       5600 non-null   float64
 4   active          5583 non-null   float64
 5   people_tested   5600 non-null   float64
 6   testing_rate    5600 non-null   float64
 7   mortality_rate  5500 non-null   float64
 8   incident_rate   5600 non-null   float64
 9   date            5590 non-null   object 
dtypes: float64(6), int64(2), object(2)
memory usage: 437.6+ KB


In [15]:
# check 'active' and fill with 0
clean_df[clean_df['active'].isnull()==True]['state'].unique()

array(['American Samoa'], dtype=object)

    Only American Samoa is missing values in active column.

In [16]:
clean_df[clean_df['state']=='American Samoa'].head()

,state,confirmed,deaths,recovered,active,people_tested,testing_rate,mortality_rate,incident_rate,date
53,American Samoa,0,0,0.0,0.0,3.0,5.391708,NaN,0.0,NaN
58,American Samoa,0,0,0.0,NaN,3.0,5.391708,NaN,0.0,NaN
114,American Samoa,0,0,0.0,NaN,3.0,5.391708,NaN,0.0,NaN
170,American Samoa,0,0,0.0,NaN,3.0,5.391708,NaN,0.0,NaN
226,American Samoa,0,0,0.0,NaN,3.0,5.391708,NaN,0.0,NaN


    The state has barely any reports (which is a good thing).
    Fill all of its nulls across all missing cols with 0 except for date.

In [17]:
clean_df['active'].fillna(0, inplace=True)
clean_df['mortality_rate'].fillna(0, inplace=True)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           5600 non-null   object 
 1   confirmed       5600 non-null   int64  
 2   deaths          5600 non-null   int64  
 3   recovered       5600 non-null   float64
 4   active          5600 non-null   float64
 5   people_tested   5600 non-null   float64
 6   testing_rate    5600 non-null   float64
 7   mortality_rate  5600 non-null   float64
 8   incident_rate   5600 non-null   float64
 9   date            5590 non-null   object 
dtypes: float64(6), int64(2), object(2)
memory usage: 437.6+ KB


In [18]:
# check states with missing dates and fill with forward fills
print(clean_df.loc[clean_df['date'].isnull()==True, 'state'].unique())

print("missing date count: ", clean_df['date'].isnull().sum())


['American Samoa' 'Virgin Islands']
missing date count:  10


In [19]:
# check if filled in
clean_df['date'] = clean_df['date'].fillna(method='ffill')

print("missing date count: {}\n".format(clean_df['date'].isnull().sum()))
clean_df.info()

missing date count: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           5600 non-null   object 
 1   confirmed       5600 non-null   int64  
 2   deaths          5600 non-null   int64  
 3   recovered       5600 non-null   float64
 4   active          5600 non-null   float64
 5   people_tested   5600 non-null   float64
 6   testing_rate    5600 non-null   float64
 7   mortality_rate  5600 non-null   float64
 8   incident_rate   5600 non-null   float64
 9   date            5600 non-null   object 
dtypes: float64(6), int64(2), object(2)
memory usage: 437.6+ KB


In [20]:
# for some reason, 'date' is no longer datetime, so convert again
clean_df['date'] = pd.to_datetime(clean_df['date'])
clean_df.head()

,state,confirmed,deaths,recovered,active,people_tested,testing_rate,mortality_rate,incident_rate,date
0,Alabama,3563,93,0.0,3470.0,21583.0,460.300152,2.610160,75.988020,2020-04-12
1,Alaska,272,8,66.0,264.0,8038.0,1344.711576,2.941176,45.504049,2020-04-12
2,Arizona,3542,115,0.0,3427.0,42109.0,578.522286,3.246753,48.662422,2020-04-12
3,Arkansas,1280,27,367.0,1253.0,19722.0,761.753354,2.109375,49.439423,2020-04-12
4,California,22795,640,0.0,22155.0,190328.0,485.423868,2.812020,58.137726,2020-04-12


In [21]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   state           5600 non-null   object        
 1   confirmed       5600 non-null   int64         
 2   deaths          5600 non-null   int64         
 3   recovered       5600 non-null   float64       
 4   active          5600 non-null   float64       
 5   people_tested   5600 non-null   float64       
 6   testing_rate    5600 non-null   float64       
 7   mortality_rate  5600 non-null   float64       
 8   incident_rate   5600 non-null   float64       
 9   date            5600 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 437.6+ KB


# Transform feature and y values (daily value diff):

1. Get daily diff for feature and target values<br><br>
    
    - aggregate each day's columns across all states. 
    - get a "safe" range and assess outliers to determine safe/risky categories
    
    * IGNORE HERE: first thought was to consider state by state, since each state has differences in volume. However, it seems to be a bit more complicated, so I am thinking to proceed with comprehensive measure across dates
    
        - get diff first for each state by agg each features across entire timeline
        try out getting daily diff for each state's mortality/incident proportion in order to compare across states
        - compare the aggregated mean of diff of each state across entire timeline state by state
        - columns as dates
        - pandas df.diff(axis=1)
            * params: 
                  periods=n (n as integer; getting diff with n prior row/col)
    

2. Assess outliers (using iqr) and group into risky/normal


* ultimately, I want to end up with a ts df where 
    columns as: dates
    groupby states with rows of daily diff of each features 
    
    - 

In [22]:
clean_df.head()

,state,confirmed,deaths,recovered,active,people_tested,testing_rate,mortality_rate,incident_rate,date
0,Alabama,3563,93,0.0,3470.0,21583.0,460.300152,2.610160,75.988020,2020-04-12
1,Alaska,272,8,66.0,264.0,8038.0,1344.711576,2.941176,45.504049,2020-04-12
2,Arizona,3542,115,0.0,3427.0,42109.0,578.522286,3.246753,48.662422,2020-04-12
3,Arkansas,1280,27,367.0,1253.0,19722.0,761.753354,2.109375,49.439423,2020-04-12
4,California,22795,640,0.0,22155.0,190328.0,485.423868,2.812020,58.137726,2020-04-12


In [218]:
dailydff = {}

for state, dfs in clean_df.groupby('state'):
    dfs_ = dfs.loc[:,['date','confirmed','deaths',
                      'recovered','active','incident_rate',
                      'people_tested','testing_rate']].set_index('date')
    for cols in dfs_.columns:
        #col_name = str(cols)
        dfs_[str(cols)+"_diff"] = round(dfs_[cols].diff(), 2)
         
    if state not in dailydff.keys():
        dailydff[state] = dfs_[[col for col in dfs_ if "_diff" in str(col)]].dropna(how='all',
                                                                                    axis=0).reset_index()
#dailydff

In [219]:
# concatenate dictionary dfs and prep to aggregate
bystate_daily = pd.concat(dailydff, axis=0)
bystate_daily.head()

date  confirmed_diff  deaths_diff  recovered_diff  \
Alabama 0 2020-04-13           171.0          6.0             0.0   
        1 2020-04-14           219.0         15.0             0.0   
        2 2020-04-15           122.0          4.0             0.0   
        3 2020-04-16           270.0         15.0             0.0   
        4 2020-04-17           226.0         15.0             0.0   

           active_diff  incident_rate_diff  people_tested_diff  \
Alabama 0        165.0                3.65              7599.0   
        1        204.0                4.67              3935.0   
        2        118.0                2.60               960.0   
        3        255.0                5.76              2314.0   
        4        211.0                4.82              1457.0   

           testing_rate_diff  
Alabama 0             162.06  
        1              83.92  
        2              20.47  
        3              49.35  
        4              31.07

In [220]:
# separate date column and unstack the states to organize as proper timeseries display
date_cols = bystate_daily['date']
states_unstack = bystate_daily.drop('date',axis=1).unstack()
states_unstack.head(10)


confirmed_diff                                          \
                                 0       1       2       3       4       5    
Alabama                       171.0   219.0   122.0   270.0   226.0   141.0   
Alaska                          5.0     8.0     8.0     7.0     9.0     5.0   
American Samoa                  0.0     0.0     0.0     0.0     0.0     0.0   
Arizona                       163.0   104.0   155.0   273.0   274.0   213.0   
Arkansas                      130.0    88.0    71.0    51.0    75.0    49.0   
California                   1136.0  1425.0  1330.0   991.0  1480.0  1334.0   
Colorado                      384.0   259.0     6.0   330.0   405.0   356.0   
Connecticut                  1346.0   608.0   766.0  1129.0   925.0   741.0   
Delaware                      133.0   168.0    88.0    56.0   247.0   221.0   
District of Columbia           80.0   103.0   139.0   153.0   126.0   190.0   

                                                     ... testing_rate_diff  \
                         6       7       8       9   ...                89   
Alabama               176.0   191.0   238.0   276.0  ...            199.95   
Alaska                  5.0     2.0     8.0     6.0  ...            197.53   
American Samoa          0.0     0.0     0.0     0.0  ...              0.00   
Arizona               209.0   135.0   188.0   217.0  ...            203.04   
Arkansas               37.0   192.0    17.0   286.0  ...            140.40   
California            940.0  2255.0  1779.0  1879.0  ...            252.98   
Colorado              683.0     0.0   743.0   418.0  ...            151.44   
Connecticut           412.0  1853.0   545.0  2109.0  ...              0.00   
Delaware                0.0   207.0   186.0   269.0  ...            144.49   
District of Columbia  127.0   134.0   171.0   108.0  ...            448.60   

                                                                              \
                          90      91      92      93      94      95      96   
Alabama               193.47  165.16  193.45  243.92  205.17  244.86  232.09   
Alaska                286.93  152.42  394.10  904.93  947.17  326.98  403.12   
American Samoa          0.00    0.00    0.00    0.00  397.19    0.00    0.00   
Arizona               167.36   85.92  268.14  192.60  182.78  215.72   96.49   
Arkansas              422.59   18.95  242.73  181.49  171.81  216.58  182.52   
California            331.30  348.67  330.51  299.45  309.35  325.45  311.60   
Colorado              108.39  126.10   97.07   87.43  108.11  141.23  111.73   
Connecticut             0.00  872.55   68.52  346.06  321.12  404.48    0.00   
Delaware              338.27  200.66  229.73  183.62  152.19  384.08  220.48   
District of Columbia  401.42  472.83  523.70  570.74  325.61  193.41  474.39   

                                      
                          97      98  
Alabama               225.12  168.36  
Alaska                635.23  352.13  
American Samoa          0.00    0.00  
Arizona               192.18  103.64  
Arkansas              204.32  452.15  
California            302.78  322.61  
Colorado               89.52  108.20  
Connecticut             0.00  766.39  
Delaware              229.83    0.00  
District of Columbia  398.16  563.23  

[10 rows x 693 columns]

In [221]:
# create time series dataset
datestr = sorted(set(str(datestrings)[0:10] for datestrings in date_cols.values))
state_stacked = states_unstack.stack(0)
state_ts = state_stacked.rename(columns=dict({state_stacked.columns[i]: datestr[i] for i in range(len(datestr))}))
print(len(datestr), len(state_stacked.columns))
state_ts.head(10)

99 99


2020-04-13  2020-04-14  2020-04-15  2020-04-16  \
Alabama active_diff             165.00      204.00      118.00      255.00   
        confirmed_diff          171.00      219.00      122.00      270.00   
        deaths_diff               6.00       15.00        4.00       15.00   
        incident_rate_diff        3.65        4.67        2.60        5.76   
        people_tested_diff     7599.00     3935.00      960.00     2314.00   
        recovered_diff            0.00        0.00        0.00        0.00   
        testing_rate_diff       162.06       83.92       20.47       49.35   
Alaska  active_diff               5.00        7.00        8.00        7.00   
        confirmed_diff            5.00        8.00        8.00        7.00   
        deaths_diff               0.00        1.00        0.00        0.00   

                            2020-04-17  2020-04-18  2020-04-19  2020-04-20  \
Alabama active_diff             211.00      136.00      172.00      185.00   
        confirmed_diff          226.00      141.00      176.00      191.00   
        deaths_diff              15.00        5.00        4.00        6.00   
        incident_rate_diff        4.82        3.01        3.75        4.07   
        people_tested_diff     1457.00     4690.00     3174.00      188.00   
        recovered_diff            0.00        0.00        0.00        0.00   
        testing_rate_diff        31.07      100.02       67.69        4.01   
Alaska  active_diff               9.00        5.00        5.00        2.00   
        confirmed_diff            9.00        5.00        5.00        2.00   
        deaths_diff               0.00        0.00        0.00        0.00   

                            2020-04-21  2020-04-22  ...  2020-07-12  \
Alabama active_diff             218.00      263.00  ...     1429.00   
        confirmed_diff          238.00      276.00  ...     1439.00   
        deaths_diff              20.00       13.00  ...       10.00   
        incident_rate_diff        5.08        5.89  ...       29.35   
        people_tested_diff     2626.00      234.00  ...     9804.00   
        recovered_diff            0.00        0.00  ...        0.00   
        testing_rate_diff        56.00        4.99  ...      199.95   
Alaska  active_diff               8.00        6.00  ...       52.00   
        confirmed_diff            8.00        6.00  ...       62.00   
        deaths_diff               0.00        0.00  ...        0.00   

                            2020-07-13  2020-07-14  2020-07-15  2020-07-16  \
Alabama active_diff            1633.00     1955.00     1670.00    -2188.00   
        confirmed_diff         1640.00     1958.00     1710.00     1812.00   
        deaths_diff               7.00        3.00       40.00       47.00   
        incident_rate_diff       33.45       39.93       34.88       36.96   
        people_tested_diff     9486.00     8098.00     9485.00    11960.00   
        recovered_diff            0.00        0.00        0.00     3953.00   
        testing_rate_diff       193.47      165.16      193.45      243.92   
Alaska  active_diff              77.00       55.00       18.00       25.00   
        confirmed_diff           94.00       60.00       40.00       52.00   
        deaths_diff               0.00        0.00        0.00        0.00   

                            2020-07-17  2020-07-18  2020-07-19  2020-07-20  \
Alabama active_diff            2002.00     1968.00     2122.00     1776.00   
        confirmed_diff         2021.00     2003.00     2143.00     1777.00   
        deaths_diff              19.00       35.00       21.00        1.00   
        incident_rate_diff       41.22       40.85       43.71       36.24   
        people_tested_diff    10060.00    12006.00    11380.00    11038.00   
        recovered_diff            0.00        0.00        0.00        0.00   
        testing_rate_diff       205.17      244.86      232.09      225.12   
Alaska  active_diff              43.00       

In [222]:
state_ts.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 392 entries, ('Alabama', 'active_diff') to ('Wyoming', 'testing_rate_diff')
Data columns (total 99 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   2020-04-13  392 non-null    float64
 1   2020-04-14  392 non-null    float64
 2   2020-04-15  392 non-null    float64
 3   2020-04-16  392 non-null    float64
 4   2020-04-17  392 non-null    float64
 5   2020-04-18  392 non-null    float64
 6   2020-04-19  392 non-null    float64
 7   2020-04-20  392 non-null    float64
 8   2020-04-21  392 non-null    float64
 9   2020-04-22  392 non-null    float64
 10  2020-04-24  392 non-null    float64
 11  2020-04-25  392 non-null    float64
 12  2020-04-26  392 non-null    float64
 13  2020-04-27  392 non-null    float64
 14  2020-04-28  392 non-null    float64
 15  2020-04-29  392 non-null    float64
 16  2020-04-30  392 non-null    float64
 17  2020-05-01  392 non-null    float64
 18  2020-05-02  392 

In [227]:
bystate_reidx = bystate_daily.reset_index(level=0).rename(columns={'level_0':'state'})
#reidx = bystate_daily.reset_index(level=1,drop=True)

bystate_reidx.head()


,state,date,confirmed_diff,deaths_diff,recovered_diff,active_diff,incident_rate_diff,people_tested_diff,testing_rate_diff
0,Alabama,2020-04-13,171.0,6.0,0.0,165.0,3.65,7599.0,162.06
1,Alabama,2020-04-14,219.0,15.0,0.0,204.0,4.67,3935.0,83.92
2,Alabama,2020-04-15,122.0,4.0,0.0,118.0,2.60,960.0,20.47
3,Alabama,2020-04-16,270.0,15.0,0.0,255.0,5.76,2314.0,49.35
4,Alabama,2020-04-17,226.0,15.0,0.0,211.0,4.82,1457.0,31.07


In [228]:
# assess by date from original data to get description of distribution
bydate_diff_avg = bystate_reidx.groupby('date')#[['confirmed_diff','incident_rate_diff']].agg(np.mean)

#bydate_diff_avg.confirmed_diff.describe()

# Assess outliers; Categorization

##### Steps:

1. Get low, high IQR bounds for each date for appropriate variables <br><br>

2. Filter out and create two new df: out-of-bounds states, normal range states <br><br>

3. using the filtered "normal" range dataset, aggregate mean for the variables (another df)<br><br>

4. with aggregated data, compute multi-linreg to select the model<br><br>

5. ts train-test split<br><br>

In [255]:
# daily distribution details and set reusable col name > check correct transform
daily_stat_confirmed, daily_stat_IR = bydate_diff_avg.confirmed_diff.describe(), bydate_diff_avg.incident_rate_diff.describe()
[
    
    
    x.rename(columns={'25%':'q1', 
                      '50%':'median', 
                      '75%':'q3'}, 
             inplace=True) for x in [daily_stat_confirmed,
                                     daily_stat_IR]
]

print("confirmed updated: \n", 
      daily_stat_confirmed.iloc[:3,1:-1])
print("\nconfirmed original describe: \n", 
      bydate_diff_avg.confirmed_diff.describe().iloc[:3,1:-1])


print("\nIR updated: \n", 
      daily_stat_IR.iloc[:3,1:-1])
print("\nIR original describe: \n",
      bydate_diff_avg.incident_rate_diff.describe().iloc[:3,1:-1])

confirmed updated: 
                   mean          std   min     q1  median      q3
date                                                            
2020-04-13  451.892857  1004.239806   0.0  20.50   100.5  423.50
2020-04-14  483.053571  1134.820089 -63.0  35.25   135.5  451.75
2020-04-15  512.142857  1561.988772   0.0  21.25   118.0  415.00

confirmed original describe: 
                   mean          std   min    25%    50%     75%
date                                                           
2020-04-13  451.892857  1004.239806   0.0  20.50  100.5  423.50
2020-04-14  483.053571  1134.820089 -63.0  35.25  135.5  451.75
2020-04-15  512.142857  1561.988772   0.0  21.25  118.0  415.00

IR updated: 
                 mean        std   min     q1  median      q3
date                                                        
2020-04-13  6.404107   8.788872 -0.00  1.315   3.255  7.2125
2020-04-14  6.930536  11.484412 -8.44  1.425   3.165  7.6625
2020-04-15  6.725357  10.629345  0.00  1.31

In [258]:
# filter: where values are < low_iqr | > high_iqr, set as risky

# 1. set "safe" low,high iqr ranges for incident_rate column (target)

q1_IR = daily_stat_IR['q1']
q3_IR = daily_stat_IR['q3']
iqr_IR = q3_IR - q1_IR
low_IR = q1_IR - 1.5 * iqr
high_IR = q3_IR + 1.5 * iqr
print("low, high bounds for y variable (incident rate) 7 day display: \n")
print("\nlowest: \n", low_IR[:7])
print("\nhighest : \n", high_IR[:7])

low, high bounds for y variable (incident rate) 7 day display: 


lowest: 
 date
2020-04-13   -603.1850
2020-04-14   -623.3250
2020-04-15   -589.3150
2020-04-16   -830.9425
2020-04-17   -864.6425
2020-04-18   -639.6850
2020-04-19   -671.6500
dtype: float64

highest : 
 date
2020-04-13    611.7125
2020-04-14    632.4125
2020-04-15    599.1400
2020-04-16    841.3600
2020-04-17    878.5425
2020-04-18    652.9125
2020-04-19    683.4075
dtype: float64


In [259]:
# 2. go back to ts data, and extract values based on the low, high iqr bounds
state_ts.head()

2020-04-13  2020-04-14  2020-04-15  2020-04-16  \
Alabama active_diff             165.00      204.00       118.0      255.00   
        confirmed_diff          171.00      219.00       122.0      270.00   
        deaths_diff               6.00       15.00         4.0       15.00   
        incident_rate_diff        3.65        4.67         2.6        5.76   
        people_tested_diff     7599.00     3935.00       960.0     2314.00   

                            2020-04-17  2020-04-18  2020-04-19  2020-04-20  \
Alabama active_diff             211.00      136.00      172.00      185.00   
        confirmed_diff          226.00      141.00      176.00      191.00   
        deaths_diff              15.00        5.00        4.00        6.00   
        incident_rate_diff        4.82        3.01        3.75        4.07   
        people_tested_diff     1457.00     4690.00     3174.00      188.00   

                            2020-04-21  2020-04-22  ...  2020-07-12  \
Alabama active_diff             218.00      263.00  ...     1429.00   
        confirmed_diff          238.00      276.00  ...     1439.00   
        deaths_diff              20.00       13.00  ...       10.00   
        incident_rate_diff        5.08        5.89  ...       29.35   
        people_tested_diff     2626.00      234.00  ...     9804.00   

                            2020-07-13  2020-07-14  2020-07-15  2020-07-16  \
Alabama active_diff            1633.00     1955.00     1670.00    -2188.00   
        confirmed_diff         1640.00     1958.00     1710.00     1812.00   
        deaths_diff               7.00        3.00       40.00       47.00   
        incident_rate_diff       33.45       39.93       34.88       36.96   
        people_tested_diff     9486.00     8098.00     9485.00    11960.00   

                            2020-07-17  2020-07-18  2020-07-19  2020-07-20  \
Alabama active_diff            2002.00     1968.00     2122.00     1776.00   
        confirmed_diff         2021.00     2003.00     2143.00     1777.00   
        deaths_diff              19.00       35.00       21.00        1.00   
        incident_rate_diff       41.22       40.85       43.71       36.24   
        people_tested_diff    10060.00    12006.00    11380.00    11038.00   

                            2020-07-21  
Alabama active_diff            1876.00  
        confirmed_diff         1880.00  
        deaths_diff               4.00  
        incident_rate_diff       38.34  
        people_tested_diff     8255.00  

[5 rows x 99 columns]

# Stopped here 07-28-2020
Work on filtering methods

In [ ]:
#state_ts.loc['Alabama','active_diff']

for m in len(range(state_ts)):
    
    #uniq_states = set(midx[0] for midx in state_ts.index)
    print(midx[1])
    
uniq_states = set(midx[0] for midx in state_ts.index)
#uniq_states